In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import sklearn as sk
import requests

Core Retrieval 

In [ ]:
import requests
import pandas as pd

CORE_API_KEY = 'ZLQojgG1uJDYRdprWS8UhEzsIPM03cNi' 

query = "natural+language+processing"
page_size = 50  # Max results per page (CORE allows up to 100)

# CORE Search API endpoint
CORE_SEARCH_URL = "https://core.ac.uk/api-v3/search/works"

CORe = f"https://core.ac.uk/search/?q={query}&page=1"
# Function to query CORE and collect results
def fetch_core_papers(query, max_pages=5):
    all_results = []

    for page in range(1, max_pages + 1):
        params = {
            'q': query,
            'page': page,
            'pageSize': page_size,
            'apiKey': CORE_API_KEY
        }

        response = requests.get(CORe, params=params)
        if response.status_code != 200:
            print(f"Error: {response.status_code} - {response.text}")
            break

        data = response.json()
        if 'data' not in data:
            print("No data found.")
            break

        for item in data['data']:
            paper_info = {
                'title': item.get('title'),
                'authors': item.get('authors'),
                'year': item.get('publishedDate', '')[:4],
                'journal': item.get('journals', [{}])[0].get('name') if item.get('journals') else None,
                'abstract': item.get('description'),
                'fullTextLink': item.get('fullTextLink')
            }
            all_results.append(paper_info)

        print(f"Page {page} fetched, total results: {len(all_results)}")

    return pd.DataFrame(all_results)

# Fetch papers and save to CSV
df = fetch_core_papers(query="machine learning", max_pages=3)
df.to_csv("core_papers_metadata.csv", index=False)
print("Saved metadata to core_papers_metadata.csv")
    

Error: 404 - <!DOCTYPE html><html lang="en"><head><meta charSet="utf-8"/><meta name="description" content="CORE is a service provided by the Knowledge Media Institute, based at The Open University, United Kingdom.
"/><link rel="shortcut icon" href="/favicon/favicon-32px.png"/><link rel="icon" href="/favicon/favicon-128px.png"/><link rel="icon" href="/favicon/favicon.svg"/><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"/><title>Page is not found</title><link rel="shortcut icon" href="/favicon/favicon-32px.png"/><link rel="icon" href="/favicon/favicon-128px.png"/><link rel="icon" href="/favicon/favicon.svg"/><meta name="next-head-count" content="10"/><link rel="preload" href="https://core.ac.uk/_next/static/css/c4c21edf3247df18ba22.css" as="style"/><link rel="stylesheet" href="https://core.ac.uk/_next/static/css/c4c21edf3247df18ba22.css" data-n-g=""/><link rel="preload" href="https://core.ac.uk/_next/static/css/69940f16912f38d7efe2.css" as="style"/><

OpenAlex retrieval


In [ ]:

import requests
import pandas as pd
import time
import urllib.parse

def fetch_openalex_papers(query: str, max_results: int = 50) -> pd.DataFrame:
    print(f"Searching OpenAlex for: {query}")
    base_url = "https://api.openalex.org/works"
    per_page = 25
    all_results = []
    cursor = "*"

    # ✅ Real, properly formatted User-Agent
    headers = {
        "User-Agent": "LLMResearchBot/1.0 (mailto:your.email@example.com)"
    }

    encoded_query = urllib.parse.quote(query)

    while len(all_results) < max_results:
        url = (
            f"{base_url}?filter=title.search:{encoded_query}"
            f"&per-page={per_page}&cursor={cursor}"
        )

        try:
            response = requests.get(url, headers=headers, timeout=10)
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            break

        if response.status_code == 403:
            print("❌ Access Forbidden (403) — Check your User-Agent and network.")
            break
        elif response.status_code != 200:
            print(f"Failed to fetch data: {response.status_code}")
            break

        data = response.json()
        results = data.get("results", [])

        for work in results:
            paper = {
                "id": work.get("id"),
                "title": work.get("title"),
                "doi": work.get("doi"),
                "publication_year": work.get("publication_year"),
                "authors": ", ".join([a['author']['display_name'] for a in work.get("authorships", [])]),
                "abstract": work.get("abstract_inverted_index", None),
                "open_access": work.get("open_access", {}).get("is_oa", False),
                "host_venue": work.get("host_venue", {}).get("display_name", "")
            }

            # Rebuild abstract from inverted index
            if paper["abstract"]:
                abstract_words = sorted(
                    [(v, k) for k, vlist in paper["abstract"].items() for v in vlist]
                )
                paper["abstract"] = " ".join([w for _, w in abstract_words])
            else:
                paper["abstract"] = ""

            all_results.append(paper)

        cursor = data.get('meta', {}).get('next_cursor', None)
        if not cursor:
            break

        time.sleep(1)  # rate-limiting

    df = pd.DataFrame(all_results)
    print(f"✅ Retrieved {len(df)} papers.")
    return df



def save_to_csv(df: pd.DataFrame, filename: str = "papers.csv"):
    df.to_csv(filename, index=False)
    print(f"Saved metadata to {filename}")


if __name__ == "__main__":
    query = input("Enter your research topic: ")
    df = fetch_openalex_papers(query, max_results=50)
    save_to_csv(df) 


Searching OpenAlex for: Large Language Model
✅ Retrieved 50 papers.
Saved metadata to papers.csv


Bidirectional Encoder Representation for Transformers(BERT)

In [1]:
from transformers import pipeline
pipe = pipeline("fill-mask", model="google-bert/bert-base-uncased") 

Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-uncased")

Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.1386,  0.1583, -0.2967,  ..., -0.2708, -0.2844,  0.4581],
         [ 0.5364, -0.2327,  0.1754,  ...,  0.5540,  0.4981, -0.0024],
         [ 0.3002, -0.3475,  0.1208,  ..., -0.4562,  0.3288,  0.8773],
         ...,
         [ 0.3799,  0.1203,  0.8283,  ..., -0.8624, -0.5957,  0.0471],
         [-0.0252, -0.7177, -0.6950,  ...,  0.0757, -0.6668, -0.3401],
         [ 0.7535,  0.2391,  0.0717,  ...,  0.2467, -0.6458, -0.3213]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.9377, -0.5043, -0.9799,  0.9030,  0.9329, -0.2438,  0.8926,  0.2288,
         -0.9531, -1.0000, -0.8862,  0.9906,  0.9855,  0.7155,  0.9455, -0.8645,
         -0.6035, -0.6666,  0.3020, -0.1587,  0.7455,  1.0000, -0.4022,  0.4261,
          0.6151,  0.9996, -0.8773,  0.9594,  0.9585,  0.6950, -0.6718,  0.3325,
         -0.9954, -0.2268, -0.9658, -0.9951,  0.6127, -0.7670,  0.0873,  0.0824,
         -0.9518,  0.4713,  1.00

In [50]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [35]:
import os
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from transformers import (
    BertModel, 
    BertTokenizerFast, 
    GPT2LMHeadModel, 
    GPT2Tokenizer,
    get_linear_schedule_with_warmup
)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")

INFO:__main__:Using device: cuda


In [12]:
torch.cuda.is_available()
print(torch.__version__)

2.7.0+cpu


In [41]:
class Config:
    BERT_MODEL_NAME = "bert-base-uncased"
    GPT_MODEL_NAME = "gpt2"
    MAX_LEN = 512
    BATCH_SIZE = 8
    LEARNING_RATE = 2e-5
    EPOCHS = 3
    WARMUP_STEPS = 0
    SAVE_PATH = "./bert_gpt_hybrid"
    TRAIN_FILE = "A:\ml_research\sample_train_data.csv"
    
    # Create directory for saving models
    os.makedirs(SAVE_PATH, exist_ok=True)

config = Config()

# Load tokenizers
bert_tokenizer = BertTokenizerFast.from_pretrained(config.BERT_MODEL_NAME)
gpt_tokenizer = GPT2Tokenizer.from_pretrained(config.GPT_MODEL_NAME)
if gpt_tokenizer.pad_token is None:
    gpt_tokenizer.pad_token = gpt_tokenizer.eos_token

<>:10: SyntaxWarning: invalid escape sequence '\m'
<>:10: SyntaxWarning: invalid escape sequence '\m'
C:\Users\aravi\AppData\Local\Temp\ipykernel_31296\833307183.py:10: SyntaxWarning: invalid escape sequence '\m'
  TRAIN_FILE = "A:\ml_research\sample_train_data.csv"


In [42]:
class TextDataset(Dataset):
    def __init__(self, texts, targets, bert_tokenizer, gpt_tokenizer, max_len):
        self.texts = texts
        self.targets = targets
        self.bert_tokenizer = bert_tokenizer
        self.gpt_tokenizer = gpt_tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        target = str(self.targets[idx])
        
        # Tokenize for BERT
        bert_inputs = self.bert_tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        
        # Tokenize for GPT2
        gpt_inputs = self.gpt_tokenizer(
            target,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        
        return {
            "bert_input_ids": bert_inputs["input_ids"].squeeze(),
            "bert_attention_mask": bert_inputs["attention_mask"].squeeze(),
            "gpt_input_ids": gpt_inputs["input_ids"].squeeze(),
            "gpt_attention_mask": gpt_inputs["attention_mask"].squeeze(),
            "target_text": target
        }


In [43]:
class BertGPTHybrid(nn.Module):
    def __init__(self, bert_model_name, gpt_model_name):
        super(BertGPTHybrid, self).__init__()
        # Load pre-trained models
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.gpt = GPT2LMHeadModel.from_pretrained(gpt_model_name)
        
        # Adapter layer between BERT and GPT2
        bert_hidden_size = self.bert.config.hidden_size
        gpt_hidden_size = self.gpt.config.n_embd
        
        self.adapter = nn.Sequential(
            nn.Linear(bert_hidden_size, gpt_hidden_size),
            nn.LayerNorm(gpt_hidden_size),
            nn.Dropout(0.1),
            nn.ReLU()
        )
        
    def forward(self, bert_input_ids, bert_attention_mask, gpt_input_ids=None, gpt_attention_mask=None):
        # Get BERT embeddings
        bert_outputs = self.bert(
            input_ids=bert_input_ids,
            attention_mask=bert_attention_mask,
            return_dict=True
        )
        
        # Get the [CLS] token representation which summarizes the input
        bert_cls_output = bert_outputs.last_hidden_state[:, 0, :]
        
        # Adapt BERT output to GPT2 input dimensions
        adapted_output = self.adapter(bert_cls_output)
        
        # Reshape to match what GPT2 expects for embeddings
        # This becomes our "prompt" embedding for GPT2
        batch_size = adapted_output.shape[0]
        adapted_output = adapted_output.unsqueeze(1)  # Shape: [batch_size, 1, gpt_hidden_size]
        
        # Set up for GPT2 generation
        if gpt_input_ids is not None:
            # Training mode - we have targets
            gpt_outputs = self.gpt(
                input_ids=gpt_input_ids,
                attention_mask=gpt_attention_mask,
                labels=gpt_input_ids,
                return_dict=True
            )
            
            # Extract the loss and logits
            loss = gpt_outputs.loss
            logits = gpt_outputs.logits
            
            return {
                "loss": loss,
                "logits": logits
            }
        else:
            # Inference mode - generate text
            # We use the adapted BERT output to condition the GPT2 generation
            # This is a simplified approach and can be extended
            return self.gpt.generate(
                input_embeddings=adapted_output,
                max_length=50,
                num_return_sequences=1
            )

In [44]:
def load_and_prepare_data(filepath):
    df = pd.read_csv(filepath)
    
    # Assuming the CSV has 'input' and 'output' columns
    train_texts, val_texts, train_targets, val_targets = train_test_split(
        df['input'].values, df['output'].values, test_size=0.1, random_state=42
    )
    
    # Create datasets
    train_dataset = TextDataset(
        texts=train_texts,
        targets=train_targets,
        bert_tokenizer=bert_tokenizer,
        gpt_tokenizer=gpt_tokenizer,
        max_len=config.MAX_LEN
    )
    
    val_dataset = TextDataset(
        texts=val_texts,
        targets=val_targets,
        bert_tokenizer=bert_tokenizer,
        gpt_tokenizer=gpt_tokenizer,
        max_len=config.MAX_LEN
    )
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=config.BATCH_SIZE,
        shuffle=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=config.BATCH_SIZE,
        shuffle=False
    )
    
    return train_loader, val_loader


In [45]:
def train_model(model, train_loader, val_loader, config):
    # Set up optimizer
    optimizer = AdamW(model.parameters(), lr=config.LEARNING_RATE)
    
    # Calculate total steps for scheduler
    total_steps = len(train_loader) * config.EPOCHS
    
    # Create scheduler
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=config.WARMUP_STEPS,
        num_training_steps=total_steps
    )
    
    # Training loop
    for epoch in range(config.EPOCHS):
        logger.info(f"Starting epoch {epoch + 1}/{config.EPOCHS}")
        
        # Training phase
        model.train()
        train_losses = []
        
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}"):
            # Move batch to device
            bert_input_ids = batch["bert_input_ids"].to(device)
            bert_attention_mask = batch["bert_attention_mask"].to(device)
            gpt_input_ids = batch["gpt_input_ids"].to(device)
            gpt_attention_mask = batch["gpt_attention_mask"].to(device)
            
            # Forward pass
            outputs = model(
                bert_input_ids=bert_input_ids,
                bert_attention_mask=bert_attention_mask,
                gpt_input_ids=gpt_input_ids,
                gpt_attention_mask=gpt_attention_mask
            )
            
            loss = outputs["loss"]
            train_losses.append(loss.item())
            
            # Backward pass
            loss.backward()
            
            # Gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            # Update weights
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        
        # Calculate average training loss
        train_loss = np.mean(train_losses)
        logger.info(f"Epoch {epoch + 1} - Training loss: {train_loss:.4f}")
        
        # Validation phase
        model.eval()
        val_losses = []
        
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Validation Epoch {epoch + 1}"):
                # Move batch to device
                bert_input_ids = batch["bert_input_ids"].to(device)
                bert_attention_mask = batch["bert_attention_mask"].to(device)
                gpt_input_ids = batch["gpt_input_ids"].to(device)
                gpt_attention_mask = batch["gpt_attention_mask"].to(device)
                
                # Forward pass
                outputs = model(
                    bert_input_ids=bert_input_ids,
                    bert_attention_mask=bert_attention_mask,
                    gpt_input_ids=gpt_input_ids,
                    gpt_attention_mask=gpt_attention_mask
                )
                
                loss = outputs["loss"]
                val_losses.append(loss.item())
        
        # Calculate average validation loss
        val_loss = np.mean(val_losses)
        logger.info(f"Epoch {epoch + 1} - Validation loss: {val_loss:.4f}")
        
        # Save model checkpoint
        checkpoint_path = os.path.join(config.SAVE_PATH, f"checkpoint_epoch_{epoch + 1}.pt")
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss
        }, checkpoint_path)
        logger.info(f"Model checkpoint saved to {checkpoint_path}")
    
    # Save the final model
    final_model_path = os.path.join(config.SAVE_PATH, "final_model.pt")
    torch.save(model.state_dict(), final_model_path)
    logger.info(f"Final model saved to {final_model_path}")
    
    return model


In [46]:
# Generation function for testing
def generate_text(model, input_text, bert_tokenizer, gpt_tokenizer, max_len=100):
    model.eval()
    
    # Tokenize input text for BERT
    bert_inputs = bert_tokenizer(
        input_text,
        max_length=config.MAX_LEN,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    
    # Move to device
    bert_input_ids = bert_inputs["input_ids"].to(device)
    bert_attention_mask = bert_inputs["attention_mask"].to(device)
    
    # Get BERT embeddings
    with torch.no_grad():
        bert_outputs = model.bert(
            input_ids=bert_input_ids,
            attention_mask=bert_attention_mask,
            return_dict=True
        )
    
    # Get the [CLS] token representation
    bert_cls_output = bert_outputs.last_hidden_state[:, 0, :]
    
    # Adapt BERT output to GPT2 input dimensions
    adapted_output = model.adapter(bert_cls_output)
    adapted_output = adapted_output.unsqueeze(1)
    
    # Generate text
    with torch.no_grad():
        output_ids = model.gpt.generate(
            inputs_embeds=adapted_output,
            max_length=max_len,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.2,
            do_sample=True,
            num_return_sequences=1
        )
    
    # Decode output tokens
    output_text = gpt_tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    return output_text

In [52]:
def main():
    logger.info("Loading data...")
    train_loader, val_loader = load_and_prepare_data(config.TRAIN_FILE)
    
    logger.info("Initializing model...")
    model = BertGPTHybrid(
        bert_model_name=config.BERT_MODEL_NAME,
        gpt_model_name=config.GPT_MODEL_NAME
    )
    model = model.to('cpu')
    
    logger.info("Starting training...")
    model = train_model(model, train_loader, val_loader, config)
    
    # Example of how to use the model for inference
    logger.info("Testing model with sample input...")
    sample_input = "This is a test input for our BERT-GPT hybrid model."
    generated_text = generate_text(model, sample_input, bert_tokenizer, gpt_tokenizer)
    logger.info(f"Input: {sample_input}")
    logger.info(f"Generated: {generated_text}")
    
    logger.info("Training and evaluation completed!")

if __name__ == "__main__":
    main()

INFO:__main__:Loading data...
INFO:__main__:Initializing model...
INFO:__main__:Starting training...
INFO:__main__:Starting epoch 1/3
Training Epoch 1:   0%|          | 0/12 [00:00<?, ?it/s]


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [3]:
import torch
torch.cuda.empty_cache()